<font size = 6> <b> Lec10.모델 앙상블

In [1]:
import numpy as np
import pandas as pd

# Data Load
* df.info()
* df.shape
* df.head()

In [2]:
df = pd.read_csv("lec10_가공완료.titanic.csv")
df

,survived,pclass,sex,age,fare,name_title,family,fare_per_family,cabin2,etc
0,0,3,0,2,7.2500,0,1,3.62500,0,0
1,1,1,1,3,71.2833,1,1,35.64165,1,1
2,1,3,1,2,7.9250,2,0,7.92500,0,0
3,1,1,1,3,53.1000,1,1,26.55000,1,1
4,0,3,0,3,8.0500,0,0,8.05000,0,0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,2,13.0000,0,0,13.00000,0,0
887,1,1,1,1,30.0000,2,0,30.00000,2,1
888,0,3,1,2,23.4500,2,3,5.86250,0,0
889,1,1,0,2,30.0000,0,0,30.00000,1,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   survived         891 non-null    int64  
 1   pclass           891 non-null    int64  
 2   sex              891 non-null    int64  
 3   age              891 non-null    int64  
 4   fare             891 non-null    float64
 5   name_title       891 non-null    int64  
 6   family           891 non-null    int64  
 7   fare_per_family  891 non-null    float64
 8   cabin2           891 non-null    int64  
 9   etc              891 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 69.7 KB


# 1차점수
* 트리 : DecisionTreeClassifier()

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

y = df["survived"]
X = df.drop('survived', axis=1)

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=13, stratify = y)
model = DecisionTreeClassifier(random_state=13)
# model = GradientBoostingClassifier(random_state=13)
model.fit(train_X, train_y)
pred = model.predict(test_X)

# ----------------------------------- 학습 1회 점수 -------------------------
accuracy = accuracy_score(test_y, pred)
f1 = f1_score(test_y, pred)
# ------------------------------------- 교차검증 ----------------------------
skfold = StratifiedKFold(n_splits = 5 ,shuffle=True, random_state=13)
cvs = cross_val_score(model, X, y, scoring="f1", cv=skfold)


print(f"accuracy : {accuracy:.4f},\t f1 : {f1:.4f}, \t cross_val_score : {cvs.mean():.4f}")

accuracy : 0.7836,	 f1 : 0.7364, 	 cross_val_score : 0.7239


# 2차 점수
* 트리 : DecisionTreeClassifier()
* 부스팅 : GradientBoostingClassifier(), XGBClassifier() 

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

y = df["survived"]
X = df.drop('survived', axis=1)

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=13, stratify = y)
models = [DecisionTreeClassifier(), GradientBoostingClassifier(), XGBClassifier()] #, LGBMClassifier() ]
for model in models:
    model.fit(train_X, train_y)
    pred = model.predict(test_X)

    # --------------------------------- 학습 1회 점수 ---------------------------
    accuracy = accuracy_score(test_y, pred)
    f1 = f1_score(test_y, pred)
    # ------------------------------------- 교차검증 ----------------------------
    skfold = StratifiedKFold(n_splits = 3 ,shuffle=True, random_state=13)
    cvs = cross_val_score(model, X, y, scoring="f1", cv=skfold)
        
    print(f"{model.__class__.__name__}  accuracy : {accuracy:.4f},\t f1 : {f1:.4f}, \t cross_val_score : {cvs.mean():.4f}")

DecisionTreeClassifier  accuracy : 0.7724,	 f1 : 0.7215, 	 cross_val_score : 0.7332
GradientBoostingClassifier  accuracy : 0.8284,	 f1 : 0.7850, 	 cross_val_score : 0.7676
XGBClassifier  accuracy : 0.7799,	 f1 : 0.7204, 	 cross_val_score : 0.7638


# 3차 점수
* 배깅 : RandomForestClassifier()

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier(random_state=111)
X = df.drop('survived', axis=1)
y = df['survived']
# max_depth=None, min_samples_split=2, min_samples_leaf=1
myparam = {"n_estimators" : [150, 300],
          "max_depth" : [10,20],                 # myparam에 기입된 값만 학습에 활용
          "min_samples_split" : [1,2,3],
          "min_samples_leaf" : [1,2,3]
          }

skfold = StratifiedKFold(n_splits=3, shuffle =True, random_state=33)
gcv_model = GridSearchCV(model, param_grid=myparam, scoring="f1_macro", refit=True, cv = skfold)
gcv_model.fit(X, y)

print(gcv_model.best_estimator_)
print(gcv_model.best_score_)
print(gcv_model.best_params_)

C:\AI\pythonProject\venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
36 fits failed out of a total of 108.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "C:\AI\pythonProject\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\AI\pythonProject\venv\lib\site-packages\sklearn\ensemble\_forest.py", line 340, in fit
    self._validate_params()
  File "C:\AI\pythonProject\venv\lib\site-packages\sklearn\base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "C:\AI\pythonProject\venv\lib\site-packag

RandomForestClassifier(max_depth=10, n_estimators=300, random_state=111)
0.8293194127149567
{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


# 4차 점수
* 앙상블

```python
class sklearn.ensemble.VotingClassifier(estimators, *, voting='hard', weights=None, n_jobs=None, flatten_transform=True, verbose=False)
```

In [7]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegression


lr = LogisticRegression(random_state=333)
rf = RandomForestClassifier(max_depth = 10, n_estimators=300, random_state=333)
gbc = GradientBoostingClassifier(loss='log_loss', learning_rate=0.1 ,n_estimators=20, random_state=333)
xgb = XGBClassifier()

X = df.drop('survived', axis=1)
y = df['survived']


model_list = [("lr", lr)
             ,("rf", rf)
             ,("gbc", gbc )
             ,("xgboost", xgb )
             ]

vc_model = VotingClassifier(estimators=model_list, voting='soft')
vc_model.fit(train_X, train_y)
pred = vc_model.predict(test_X)
# ---------------------- 학습 1회 점수 ------------------------------------
accuracy = accuracy_score(test_y, pred)
f1 = f1_score(test_y, pred)
print(f"앙상블 accuracy: {accuracy:.4f}\t f1:{f1:.4f}")

for val in model_list:
    model = val[1]
    model_name = val[0]
    model.fit(train_X, train_y)
    pred = model.predict(test_X)
    # ---------------------- 학습 1회 점수 ------------------------------------
    accuracy = accuracy_score(test_y, pred)
    f1 = f1_score(test_y, pred)
    # ------------------------------------- 교차검증 ----------------------------
    # skfold = StratifiedKFold(n_splits = 5 ,shuffle=True, random_state=13)
    # cvs = cross_val_score(model, X, y, scoring="f1", cv=skfold)
          
    print(f"{model.__class__.__name__} accuracy : {accuracy:.4f},\t f1 : {f1:.4f}") #, \t cross_val_score : {cvs.mean():.4f}")

C:\AI\pythonProject\venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


앙상블 accuracy: 0.8246	 f1:0.7751
LogisticRegression accuracy : 0.8022,	 f1 : 0.7558


C:\AI\pythonProject\venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RandomForestClassifier accuracy : 0.8209,	 f1 : 0.7736
GradientBoostingClassifier accuracy : 0.8246,	 f1 : 0.7773
XGBClassifier accuracy : 0.7799,	 f1 : 0.7204


# pycaret

* ref : https://pycaret.org/
* ref : https://pycaret.gitbook.io/docs/get-started/quickstart    
* ref : https://pycaret.gitbook.io/docs/get-started/functions    

* <font color=red size=4><b>[3.전처리] </b></font>  
    s = <font color=blue size=2><b>setup</b></font>(data=df, target='col')

In [8]:
# train_size : float, default = 0.7
# fold : default = 10
# fold_shuffle : default = False
from pycaret.classification import *
s = setup(df, target = 'survived', train_size=0.8, fold=3, fold_shuffle=True, session_id=1212)  # , silent=True)

,Description,Value
0,Session id,1212
1,Target,survived
2,Target type,Binary
3,Original data shape,"(891, 10)"
4,Transformed data shape,"(891, 10)"
5,Transformed train set shape,"(712, 10)"
6,Transformed test set shape,"(179, 10)"
7,Numeric features,9
8,Preprocess,True
9,Imputation type,simple


* <font color=red size=4><b>[4.학습] </b></font>  
    best_model = <font color=blue size=2><b>compare_models</b></font>(..)  
    dt = <font color=blue size=2><b>create_model</b></font>('dt')  

In [9]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [10]:
models(internal=True)

,Name,Reference,Turbo,Special,Class,Equality,Args,Tune Grid,Tune Distributions,Tune Args,SHAP,GPU Enabled,Boosting Supported,Soft Voting,Tunable Class
ID,,,,,,,,,,,,,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True,False,<class 'sklearn.linear_model._logistic.Logisti...,<function ModelContainer.__init__.<locals>.<la...,"{'max_iter': 1000, 'random_state': 1212}","{'C': [0.001, 0.002, 0.003, 0.004, 0.005, 0.00...","{'C': UniformDistribution(lower=0.001, upper=1...",{},False,False,True,True,None
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True,False,<class 'sklearn.neighbors._classification.KNei...,<function ModelContainer.__init__.<locals>.<la...,{'n_jobs': -1},"{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10...",{'n_neighbors': IntUniformDistribution(lower=1...,{},False,False,True,True,None
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True,False,<class 'sklearn.naive_bayes.GaussianNB'>,<function ModelContainer.__init__.<locals>.<la...,{},"{'var_smoothing': [1e-09, 2e-09, 5e-09, 8e-09,...",{'var_smoothing': UniformDistribution(lower=1e...,{},False,False,True,True,None
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True,False,<class 'sklearn.tree._classes.DecisionTreeClas...,<function ModelContainer.__init__.<locals>.<la...,{'random_state': 1212},"{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ...","{'max_depth': IntUniformDistribution(lower=1, ...",{},type1,False,True,True,None
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True,False,<class 'sklearn.linear_model._stochastic_gradi...,<function ModelContainer.__init__.<locals>.<la...,"{'tol': 0.001, 'loss': 'hinge', 'penalty': 'l2...","{'penalty': ['elasticnet', 'l2', 'l1'], 'l1_ra...","{'l1_ratio': UniformDistribution(lower=1e-10, ...",{},False,False,False,False,None
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False,False,<class 'sklearn.svm._classes.SVC'>,<function ModelContainer.__init__.<locals>.<la...,"{'gamma': 'auto', 'C': 1.0, 'probability': Tru...","{'C': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06...","{'C': UniformDistribution(lower=0, upper=50, l...",{},False,False,False,False,None
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False,False,<class 'sklearn.gaussian_process._gpc.Gaussian...,<function ModelContainer.__init__.<locals>.<la...,"{'copy_X_train': False, 'random_state': 1212, ...","{'max_iter_predict': [100, 200, 300, 400, 500,...",{'max_iter_predict': IntUniformDistribution(lo...,{},False,False,True,True,None
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False,False,<class 'sklearn.neural_network._multilayer_per...,<function ModelContainer.__init__.<locals>.<la...,"{'random_state': 1212, 'max_iter': 500}","{'learning_rate': ['constant', 'invscaling', '...","{'alpha': UniformDistribution(lower=1e-10, upp...",{},False,False,True,True,<class 'pycaret.internal.tunable.TunableMLPCla...
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True,False,<class 'sklearn.linear_model._ridge.RidgeClass...,<function ModelContainer.__init__.<locals>.<la...,{'random_state': 1212},"{'alpha': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06,...","{'alpha': UniformDistribution(lower=0.001, upp...",{},False,False,False,False,None


## 모델 비교
<pre>
<font color=red>
* sort                  : 'mae'
* include/exclude       : ['lr', 'dt', 'lightgbm']
* n_select              : 3
* probability_threshold : 0.25 (이진분류 임계값)
* cross_validation      : False


### compare_models()

In [11]:
# best_model = compare_models()
# best_model = compare_models(include = ['lr','rf']    , sort='MAE') 
best_model = compare_models(exclude = ['catboost'] , n_select=3, sort='F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8174,0.8648,0.7289,0.7824,0.7536,0.6090,0.6110,0.0900
lightgbm,Light Gradient Boosting Machine,0.8118,0.8506,0.7436,0.7603,0.7518,0.6003,0.6005,0.0700
xgboost,Extreme Gradient Boosting,0.8090,0.8460,0.7326,0.7610,0.7463,0.5933,0.5938,0.0700
rf,Random Forest Classifier,0.8020,0.8470,0.7070,0.7606,0.7326,0.5757,0.5769,0.1100
ridge,Ridge Classifier,0.7977,0.0000,0.7179,0.7479,0.7320,0.5697,0.5706,0.3567
lda,Linear Discriminant Analysis,0.7977,0.8500,0.7179,0.7479,0.7320,0.5697,0.5706,0.0567
lr,Logistic Regression,0.7991,0.8502,0.7106,0.7549,0.7314,0.5712,0.5726,0.5533
ada,Ada Boost Classifier,0.7991,0.8403,0.7106,0.7554,0.7309,0.5711,0.5731,0.0833
dt,Decision Tree Classifier,0.7739,0.7828,0.7033,0.7064,0.7044,0.5214,0.5218,0.4400
et,Extra Trees Classifier,0.7753,0.8314,0.6850,0.7158,0.6999,0.5206,0.5210,0.1033


In [12]:
print(best_model)

[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=1212, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1212, reg_alpha=0.0, reg_lambda=0.0, sub

### create_model()

```python
RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
```

In [13]:
# ------------------- 모델 생성
rf = create_model('rf'
                  , n_estimators = 200
                  , max_depth = 10
                  , min_samples_leaf=1
                  , min_samples_split=2
                  , cross_validation = True
                 # , return_train_score = True
                 )
print(rf)
print("=="*30)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8235,0.8669,0.7033,0.8101,0.7529,0.6167,0.6205
1,0.8101,0.8469,0.7253,0.7674,0.7458,0.5944,0.5950
2,0.8228,0.8422,0.7363,0.7882,0.7614,0.6207,0.6216
Mean,0.8188,0.8520,0.7216,0.7886,0.7534,0.6106,0.6124
Std,0.0062,0.0107,0.0137,0.0174,0.0064,0.0116,0.0123


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=-1, oob_score=False,
                       random_state=1212, verbose=0, warm_start=False)


* <font color=red size=4><b>[5.내장기능]</b></font>     
    <font color=blue size=2><b>pull()</b></font> : 점수보기    
    <font color=blue size=2><b>models()</b></font>  : 모델보기  
    <font color=blue size=2><b>get/set_config()</b></font> : setup정보   
    <font color=blue size=2><b>get/add/remove_metrics()</b></font>  : 평가메트릭스  
    <font color=blue size=2><b>automl()</b></font> :  AutoML  
    


* <font color=red size=4><b>[6.최적화] </b></font>  
    tuned_model, tuner = <font color=blue size=2><b>tune_model</b></font>(dt, custom_grid=mydic)  
    ens_dt = <font color=blue size=2><b>ensemble_model</b></font>(dt, method = 'Boosting', n_estimators = 10 , choose_better = True)  
    blender = <font color=blue size=2><b>blend_models</b></font>([lr, dt, knn], fold = 5,  method = 'soft')  
    stacker = <font color=blue size=2><b>stack_models</b></font>([lr, dt, knn], meta_model=lightgbm)  
    stacker = <font color=blue size=2><b>stack_models</b></font>([lr, dt, knn], restack = False)  

In [14]:
# mydic = {'max_depth' : 10, 'min_samples_leaf' : 1, 'min_sample_split' : 2, 'n_estimators' : 300}
myparam = {"n_estimators" : [150, 200, 250],
          "max_depth" : [10,20],                 # myparam에 기입된 값만 학습에 활용
          "min_samples_split" : [1,2,3],
          "min_samples_leaf" : [1,2,3]
          }
tmodel = tune_model(rf, custom_grid=myparam)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8235,0.8694,0.7143,0.8025,0.7558,0.6184,0.6210
1,0.8228,0.8437,0.7363,0.7882,0.7614,0.6207,0.6216
2,0.8354,0.8465,0.7473,0.8095,0.7771,0.6470,0.6483
Mean,0.8273,0.8532,0.7326,0.8001,0.7648,0.6287,0.6303
Std,0.0058,0.0115,0.0137,0.0089,0.0090,0.0130,0.0128


Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [15]:
tmodel

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=3, min_weight_fraction_leaf=0.0,
                       n_estimators=250, n_jobs=-1, oob_score=False,
                       random_state=1212, verbose=0, warm_start=False)

In [16]:
tmodel = tune_model(rf)
tmodel

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8319,0.8732,0.6813,0.8493,0.7561,0.6302,0.6391
1,0.8228,0.8360,0.7033,0.8101,0.7529,0.6159,0.6196
2,0.8186,0.8489,0.7363,0.7791,0.7571,0.6125,0.6131
Mean,0.8244,0.8527,0.7070,0.8128,0.7554,0.6195,0.6239
Std,0.0056,0.0154,0.0226,0.0287,0.0018,0.0077,0.0111


Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight={},
                       criterion='gini', max_depth=8, max_features=1.0,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0, min_samples_leaf=3,
                       min_samples_split=5, min_weight_fraction_leaf=0.0,
                       n_estimators=160, n_jobs=-1, oob_score=False,
                       random_state=1212, verbose=0, warm_start=False)

In [17]:
tmodel = tune_model(rf, search_library='optuna')
tmodel

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8109,0.8583,0.7473,0.7556,0.7514,0.5988,0.5989
1,0.8101,0.8435,0.7473,0.7556,0.7514,0.5978,0.5978
2,0.8228,0.8577,0.8022,0.7526,0.7766,0.6300,0.6309
Mean,0.8146,0.8532,0.7656,0.7546,0.7598,0.6089,0.6092
Std,0.0058,0.0068,0.0259,0.0014,0.0119,0.0149,0.0153


[I 2023-09-06 17:06:48,370] Searching the best hyperparameters using 712 samples...
[I 2023-09-06 17:06:59,944] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=-1, oob_score=False,
                       random_state=1212, verbose=0, warm_start=False)

In [18]:
tmodel = tune_model(rf, search_library='scikit-optimize')
tmodel

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8235,0.8739,0.7033,0.8101,0.7529,0.6167,0.6205
1,0.8228,0.8473,0.7253,0.7952,0.7586,0.6191,0.6207
2,0.8270,0.8524,0.7692,0.7778,0.7735,0.6336,0.6336
Mean,0.8244,0.8579,0.7326,0.7944,0.7617,0.6231,0.6249
Std,0.0018,0.0115,0.0274,0.0132,0.0087,0.0074,0.0061


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='entropy',
                       max_depth=7, max_features=0.8224534830390922,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=3.902034183027504e-07,
                       min_samples_leaf=3, min_samples_split=6,
                       min_weight_fraction_leaf=0.0, n_estimators=244,
                       n_jobs=-1, oob_score=False, random_state=1212, verbose=0,
                       warm_start=False)

# 모델 변경
* 트리 : LogisticRegression, DecisionTreeClassifier
* 보팅 : VotingClassifier
* 배깅 : BaggingClassifier, RandomForestClassifier
* 부스팅 : AdaBoostClassifier, GradientBoostingClassifier
* 스태킹 : StackingClassifier
* 외부 부스팅 : XGBoost, LightGBM

In [19]:
# ! pip install xgboost
# ! pip install lightgbm
# ! pip install catboost

```python
class sklearn.linear_model.LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

class sklearn.tree.DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)

class sklearn.ensemble.VotingClassifier(estimators, *, voting='hard', weights=None, n_jobs=None, flatten_transform=True, verbose=False)

class sklearn.ensemble.BaggingClassifier(estimator=None, n_estimators=10, *, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0, base_estimator='deprecated')

class sklearn.ensemble.RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

class sklearn.ensemble.GradientBoostingClassifier(*, loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

class sklearn.ensemble.StackingClassifier(estimators, final_estimator=None, *, cv=None, stack_method='auto', n_jobs=None, passthrough=False, verbose=0)
```


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

In [21]:
lr = LogisticRegression(random_state=11)
dt = DecisionTreeClassifier(random_state=11)

voting = VotingClassifier(estimators=[ ('lr', lr), ('df', df)], voting='hard')

bagging = BaggingClassifier(estimator=dt, n_estimators=10, random_state=11)
rf = RandomForestClassifier(n_estimators=20,random_state=11)

gboost = GradientBoostingClassifier(loss='log_loss', learning_rate=0.1, n_estimators=20, random_state=11)

skfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1212)  # 분류
staking = StackingClassifier(estimators=[ ('lr', lr), ('df', df)], final_estimator=rf, cv=skfold)

### Xgboost
* https://xgboost.readthedocs.io/en/latest/python/python_api.html
* 튜닝파라미터 : https://zzinnam.tistory.com/entry/XGboost-%EC%A3%BC%EC%9A%94-%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-with-%ED%8C%8C%EC%9D%B4%EC%8D%AC

```python
XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              objective='binary:logistic', predictor=None, ...)
```

### LightGBM
* https://lightgbm.readthedocs.io/en/latest/Python-API.html
* (kaggle) : https://www.kaggle.com/code/prashant111/lightgbm-classifier-in-python
* <b>Light GBM can handle the large size of data and takes lower memory to run.</b>

```python
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
```

In [22]:
# build the lightgbm model
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(train_X, train_y)

[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000083 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 259
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=None, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

# 모델 변경
* 트리 : DecisionTreeClassifier
* 보팅 : VotingClassifier
* 배깅 : BaggingClassifier, BaggingRegressor
* 부스팅 : AdaBoostClassifier, GradientBoostingRegressor
* 스태깅 : StackingClassifier
* 외부부스팅 : XGBoost, LightGBM

In [23]:
# ! pip install xgboost
# ! pip install lightgbm
# ! pip install catboost

```python
class sklearn.linear_model.LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

class sklearn.tree.DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)

class sklearn.ensemble.VotingClassifier(estimators, *, voting='hard', weights=None, n_jobs=None, flatten_transform=True, verbose=False)

class sklearn.ensemble.BaggingClassifier(estimator=None, n_estimators=10, *, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0, base_estimator='deprecated')

class sklearn.ensemble.RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

class sklearn.ensemble.GradientBoostingClassifier(*, loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

class sklearn.ensemble.StackingClassifier(estimators, final_estimator=None, *, cv=None, stack_method='auto', n_jobs=None, passthrough=False, verbose=0)
```

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier

import xgboost as xgb
import lightgbm as lgb

from catboost import CatBoostClassifier

In [25]:
lr = LogisticRegression(random_state=13)
dt = DecisionTreeClassifier(random_state=13)

voting = VotingClassifier(estimators=[('lr', lr), ('dt', dt)], voting='hard')

bagging = BaggingClassifier(estimator = dt, n_estimators=10, random_state=13)
rf = RandomForestClassifier(n_estimators=20, random_state=13)

gboost = GradientBoostingClassifier(loss='log_loss', learning_rate=0.1, n_estimators=20, random_state=13)

skfold = StratifiedKFold(n_splits = 3 ,shuffle=True, random_state=13)
stacking = StackingClassifier(estimators=[('lr', lr), ('dt', dt)], final_estimator=rf, cv=skfold)

## Xgboost
* ref : https://xgboost.readthedocs.io/en/stable/

## Lightgbm
* ref : https://lightgbm.readthedocs.io/en/latest/Python-API.html

## catboost
* ref : https://catboost.ai/en/docs/

### Catboost
* https://catboost.ai/en/docs/

# 파이프라인

In [26]:
from sklearn.pipeline import Pipeline